In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('compas-scores-two-years.csv')

In [3]:
#data wrangling 
df['race'] = df['race'].replace('African-American', 1).replace('Caucasian', 0)

df = df[(df['race'] == 0) | (df['race'] == 1)]

df['sex'] = df['sex'].replace('Male', 1).replace('Female', 0)

df['score_text'] = df['score_text'].replace('High', 1).replace('Medium', 0).replace('Low', -1)

df['c_charge_degree'] = df['c_charge_degree'].replace('M',1).replace('F',0)

df['days_in_jail'] = (pd.to_datetime(df['c_jail_out'])-pd.to_datetime(df['c_jail_in'])).dt.days

X = df[['race','age', 'c_charge_degree', 'score_text', 'sex',
       'priors_count', 'days_b_screening_arrest', 'decile_score', 'is_recid']]
import numpy as np
y = df['two_year_recid']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( df, y, test_size=0.25, random_state=42)

X_pos = X_train[X_train['race']==1]
X_neg = X_train[X_train['race']==0]

y_pos = np.array(y_train[X_train['race']==1])
y_neg = np.array(y_train[X_train['race']==0])

col_X = ['age', 'c_charge_degree', 'score_text', 'sex',
       'priors_count', 'days_b_screening_arrest', 'decile_score', 'is_recid', 
        'days_in_jail']
X_pos = np.array(X_pos[col_X])
X_neg = np.array(X_neg[col_X])

In [4]:
#equation 12
def distances(X, v, alpha, N, P, k):
    dists = np.zeros((N, P))
    for i in range(N):
        for p in range(P):
            for j in range(k):
                dists[i, j] += (X[i, p] - v[j, p]) * (X[i, p] - v[j, p]) * alpha[p]
    return dists

In [5]:
#equation 3
def M_nk(dists, N, k):
    M_nk = np.zeros((N, k))
    exp = np.zeros((N, k))
    denom = np.zeros(N)
    for i in range(N):
        for j in range(k):
            exp[i, j] = np.exp(-1 * dists[i, j])
            denom[i] += exp[i, j]
        for j in range(k):
            if denom[i]:
                M_nk[i, j] = exp[i, j] / denom[i]
            else:
                M_nk[i, j] = exp[i, j] / 1e-6
    return M_nk

In [6]:
#equation 6
def M_k(M_nk, N, k):
    M_k = np.zeros(k)
    for j in range(k):
        for i in range(N):
            M_k[j] += M_nk[i, j]
        M_k[j] /= N
    return M_k

In [7]:
# Equation 8&9
def Loss_x(X, M_nk, v, N, D, K):
    x_n_hat = np.zeros((N, D))
    L_x = 0.0
    for n in range(N):
        for i in range(D):
            for k in range(K):
                x_n_hat[n, i] += M_nk[n, k] * v[k, i]
            L_x += (X[n, i] - x_n_hat[n, i]) * (X[n, i] - x_n_hat[n, i])
    return x_n_hat, L_x

In [8]:
#equation 11
def Loss_y(M_nk, y, w, N, k):
    yhat = np.zeros(N)
    L_y = 0.0
    for i in range(N):
        for j in range(k):
            yhat[i] += M_nk[i, j] * w[j]
        yhat[i] = 1e-6 if yhat[i] <= 0 else yhat[i]
        yhat[i] = 0.999 if yhat[i] >= 1 else yhat[i]
        L_y += -1 * y[i] * np.log(yhat[i]) - (1.0 - y[i]) * np.log(1.0 - yhat[i])
    return yhat, L_y

In [9]:
#equation 4
def LFR(params, X_pos, X_neg, y_pos, y_neg, K, A_z, A_x, A_y, results=0):   
    LFR.iters += 1
    N_pos, D = X_pos.shape # protected set
    N_neg, _ = X_neg.shape # non-protected set
    
    #print('PARAMS SHAPE: ', params.shape)
    alpha_pos = params[:D]
    alpha_neg = params[D: 2 * D]
    w = params[2 * D: (2 * D) + K]
    #print(alpha_pos.shape, alpha_neg.shape, w.shape)
    v = np.matrix(params[(2 * D) + K:]).reshape((K, D)) # set of prototypes

    dists_pos = distances(X_pos, v, alpha_pos, N_pos, D, K)
    dists_neg = distances(X_neg, v, alpha_neg, N_neg, D, K)

    M_nk_pos = M_nk(dists_pos, N_pos, K)
    M_nk_neg = M_nk(dists_neg, N_neg, K)

    M_k_pos = M_k(M_nk_pos, N_pos, K)
    M_k_neg = M_k(M_nk_neg, N_neg, K)

    L_z = 0.0
    for k in range(K):
        L_z += abs(M_k_pos[k] - M_k_neg[k]) # Equation 7

    x_n_hat_pos, L_x_pos = Loss_x(X_pos, M_nk_pos, v, N_pos, D, K)
    x_n_hat_neg, L_x_neg = Loss_x(X_neg, M_nk_neg, v, N_neg, D, K)
    L_x = L_x_pos + L_x_neg

    yhat_pos, L_y_pos = Loss_y(M_nk_pos, y_pos, w, N_pos, K)
    yhat_neg, L_y_neg = Loss_y(M_nk_neg, y_neg, w, N_neg, K)
    L_y = L_y_pos + L_y_neg

    criterion = A_x * L_x + A_y * L_y + A_z * L_z

    if results != 0:
        return yhat_pos, yhat_neg, M_nk_pos, M_nk_neg
    else:
        return criterion
LFR.iters = 0

In [10]:
import scipy
from scipy.optimize import minimize
from numpy import linalg as LA
D = X_pos.shape[1]
K = 5
alpha_pos = np.random.uniform(size=D)
alpha_neg = np.random.uniform(size=D)
w = np.random.uniform(size=K) 
v = np.random.uniform(size=K*D)
params = alpha_pos
for item in [alpha_neg, w, v]:
    params = np.append(params, item.flatten())
res = scipy.optimize.minimize(LFR, 
                        x0=params, 
                        args=(X_pos, X_neg, y_pos, y_neg, K, 1000, 1e-4, 0.1),
                        method='SLSQP')

In [11]:
D = X_pos.shape[1]
K = 5
alpha_pos = params[:D]
alpha_neg = params[D: 2*D]
w = params[2*D: (2*D) + K]
v = np.array(params[(2*D) + K: ]).reshape((K,D))

In [12]:
##calculate parity P(Y_hat = 1 | S = 0) = P(Y_hat = 1 | S = 1)

#1) segment test dataset into sensitive and non sensitive
X_pos = X_test[X_test['race']==1]
X_neg = X_test[X_test['race']==0]

y_pos = np.array(y_test[X_test['race']==1])
y_neg = np.array(y_test[X_test['race']==0])

col_X = ['age', 'c_charge_degree', 'score_text', 'sex',
       'priors_count', 'days_b_screening_arrest', 
       'decile_score', 'days_in_jail', 'is_recid']

X_pos = np.array(X_pos[col_X])
X_neg = np.array(X_neg[col_X])

#2) calculate Y_hat for neg & pos
#recover M_nk
N_pos, D = X_pos.shape # protected set
N_neg, _ = X_neg.shape # non-protected set


dists_pos = distances(X_pos, v, alpha_pos, N_pos, D, K)
dists_neg = distances(X_neg, v, alpha_neg, N_neg, D, K)

M_nk_pos = M_nk(dists_pos, N_pos, K)
M_nk_neg = M_nk(dists_neg, N_neg, K)

yhat_pos, lypos = Loss_y(M_nk_pos, y_pos, w, N_pos, K)
yhat_neg, lyneg = Loss_y(M_nk_neg, y_neg, w, N_neg, K)

#3) find y_hat = 1 for both neg and positive
"""FINDING PARITY"""
(len(yhat_neg[yhat_neg > 0.5])/len(yhat_neg), #for non-sensitive attribute #0.584070796460177
  len(yhat_pos[yhat_pos > 0.5])/len(yhat_pos)) #for sensitive attribute #0.6409774436090225

(0.23289902280130292, 0.6277056277056277)

In [13]:
"""evaluation metric 4: calibration"""

calibration_pos = len(yhat_pos[((yhat_pos > 0.5) & (y_pos > 0.5)) |
    ((yhat_pos <= 0.5) & (y_pos <= 0.5))])/len(yhat_pos)
calibration_neg = len(yhat_neg[((yhat_neg > 0.5) & (y_neg > 0.5)) |
    ((yhat_neg <= 0.5) & (y_neg <= 0.5))])/len(yhat_neg)
calibration_pos, calibration_neg   

(0.4253246753246753, 0.5586319218241043)